# **Segundo dataset: 2da parte de la limpieza**

# **1. Importación del dataset de carpetas de la FGJ ya filtrado para delitos sobre la libertad personal y su breve exploración:**

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns

secuestros=pd.read_csv("Carpetas_Secuestros_FGJ_1.csv", encoding='utf-8')
display(secuestros.head(2))
print("-----------------------------------")
print(secuestros.shape)
print("-----------------------------------")
print(secuestros.info())
print("-----------------------------------")
print(secuestros.isna().any())
print("-----------------------------------")
print(secuestros.columns)
print("-----------------------------------")
print(secuestros.isna().sum())
print("-----------------------------------")
print(pd.unique(secuestros['alcaldia_hecho']))
print("-----------------------------------")
print(f'El número de valores faltantes en la columna "alcaldia_hecho" es: {secuestros['alcaldia_hecho'].isna().sum()}')

,anio_inicio,mes_inicio,fecha_inicio,fecha_hecho,hora_hecho,delito,categoria_delito,fiscalia,agencia,unidad_investigacion,colonia_catalogo,alcaldia_hecho
0,2016,Enero,2016-01-20,2016-01-19,20:30:00,SECUESTRO EXPRESS (PARA COMETER ROBO O EXTORSIÓN),DELITO DE BAJO IMPACTO,INVESTIGACIÓN EN BENITO JUÁREZ,BJ-1,UI-1SD,Santa Maria Nonoalco,BENITO JUAREZ
1,2016,Febrero,2016-02-03,2016-01-28,23:00:00,SECUESTRO EXPRESS (PARA COMETER ROBO O EXTORSIÓN),DELITO DE BAJO IMPACTO,INVESTIGACIÓN EN AGENCIAS DE ATENCIÓN ESPECIAL...,STCMTS,UI-1SD,Buenavista,CUAUHTEMOC


-----------------------------------
(6703, 12)
-----------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6703 entries, 0 to 6702
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   anio_inicio           6703 non-null   int64 
 1   mes_inicio            6703 non-null   object
 2   fecha_inicio          6702 non-null   object
 3   fecha_hecho           6698 non-null   object
 4   hora_hecho            6699 non-null   object
 5   delito                6703 non-null   object
 6   categoria_delito      6703 non-null   object
 7   fiscalia              6702 non-null   object
 8   agencia               6703 non-null   object
 9   unidad_investigacion  6701 non-null   object
 10  colonia_catalogo      6128 non-null   object
 11  alcaldia_hecho        6490 non-null   object
dtypes: int64(1), object(11)
memory usage: 628.5+ KB
None
-----------------------------------
anio_inicio     

# **2. Proceso de imputación de valores faltantes para la alcaldía del hecho:**

Las columnas 'agencia' y 'fiscalia' contienen información cercana a la columna 'alcaldia_hecho', por lo que serán útiles para el proceso de imputación.

La columna 'fiscalia' es muy específica cuando se trata de indicar alcaldías, sin embargo, vamos también a explorar la columna 'agencia' para ver qué tan útil nos es para la imputación de valores faltantes.

### **2.1 Imputación de valores NaN en 'alcaldia_hecho' mediante los datos de 'fiscalia':**

Expresiones regulares posibles que corresponden a las alcaldías en la columna 'fiscalia', considerando los posibles errores asociados a los caracteres especiales como las tildes:

Álvaro Obregón-LVARO, Azcapotzalco-AZCAP,  Benito Juárez-BENITO, Coyoacán-COYO, Cuajimalpa de Morelos-CUAJI, Cuauhtémoc-CUAU, Gustavo A. Madero-MADERO, Iztacalco-IZTACALCO, Iztapalapa-IZTAPALAPA, La Magdalena Contreras-MAGDALENA, Miguel Hidalgo-MIGUEL, Milpa Alta-MILPA, Tláhuac-HUAC, Tlalpan-TLALPAN, Venustiano Carranza-VENUS y Xochimilco-XOCHI.

In [2]:
#Individualmente:
#Álvaro Obregón
#secuestros.loc[secuestros['fiscalia'].str.contains('LVARO', case=False, na=False) & secuestros['alcaldia_hecho'].isna(),'alcaldia_hecho'] = 'ALVARO OBREGON'
#print(f'El número de valores faltantes en la columna "alcaldia_hecho" es: {secuestros['alcaldia_hecho'].isna().sum()}')


#En un ciclo for:
REGEX_alcaldias={
    'LVARO': 'ALVARO OBREGON', 
    'AZCAP': 'AZCAPOTZALCO',
    'BENITO': 'BENITO JUAREZ', 
    'COYOAC': 'COYOACAN',
    'CUAJI': 'CUAJIMALPA DE MORELOS', 
    'CUAU': 'CUAUHTEMOC',
    'MADERO': 'GUSTAVO A. MADERO',
    'IZTACALCO': 'IZTACALCO',
    'IZTAPALAPA': 'IZTAPALAPA',
    'MAGDALENA': 'LA MAGDALENA CONTRERAS',
    'MIGUEL': 'MIGUEL HIDALGO',
    'MILPA': 'MILPA ALTA',
    'HUAC': 'TLAHUAC',
    'TLALPAN': 'TLALPAN',
    'VENUS': 'VENUSTIANO CARRANZA',
    'XOCHI': 'XOCHIMILCO'}

for argumento_contains, alcaldia in REGEX_alcaldias.items():
    secuestros.loc[(secuestros['fiscalia'].str.contains(argumento_contains, case=False, na=False) & secuestros['alcaldia_hecho'].isna()), 'alcaldia_hecho'] = alcaldia

print(f'Ahora el número de valores faltantes en la columna "alcaldia_hecho" es: {secuestros['alcaldia_hecho'].isna().sum()}')

Ahora el número de valores faltantes en la columna "alcaldia_hecho" es: 109


### **2.2 Imputación de valores NaN en 'alcaldia_hecho' mediante los datos de 'agencia':**

In [3]:
AlcaldiaHecho_NaN=secuestros[pd.isna(secuestros['alcaldia_hecho'])] 
print("-----------------------------------")
print(pd.unique(AlcaldiaHecho_NaN['fiscalia']))
print("-----------------------------------")
print(pd.unique(AlcaldiaHecho_NaN['agencia']))
print("-----------------------------------")
print(AlcaldiaHecho_NaN.columns)
print("-----------------------------------")
print(AlcaldiaHecho_NaN.shape)

-----------------------------------
['INVESTIGACIÓN PARA LA ATENCIÓN DEL DELITO DE SECUESTRO'
 'INVESTIGACIÓN DE LOS DELITOS COMETIDOS POR SERVIDORES PUBLICOS'
 'FISCALÍA DE INVESTIGACIÓN DEL DELITO DE SECUESTRO'
 'INVESTIGACIÓN PARA LA ATENCIÓN DE NIÑOS, NIÑAS Y ADOLESCENTES'
 'INVESTIGACIÓN EN AGENCIAS DE ATENCIÓN ESPECIALIZADAS'
 'JUZGADOS FAMILIARES' 'AGENCIA CENTRAL DE INVESTIGACIÓN'
 'INVESTIGACIÓN PARA LA ATENCIÓN DEL DELITO DE HOMICIDIO'
 'FISCALÍA DE INVESTIGACIÓN DE DELITOS COMETIDOS EN AGRAVIO DE NIÑAS, NIÑOS Y ADOLESCENTES'
 'FISCALÍA DE INVESTIGACIÓN Y PERSECUCIÓN DE LOS DELITOS EN MATERIA DE DESAPARICIÓN FORZADA DE PERSONAS Y LA DESAPARICIÓN COMETIDA POR PARTICULARES Y BÚSQUEDA DE PERSONAS DESAPARECIDAS'
 'FISCALÍA DE INVESTIGACIÓN ESTRATÉGICA DE ASUNTOS ESPECIALES'
 'FISCALÍA DE INVESTIGACIÓN DE DELITOS COMETIDOS EN AGRAVIO DE GRUPOS DE ATENCIÓN PRIORITARIA'
 'FISCALÍA PARA LA INVESTIGACIÓN DE LOS DELITOS COMETIDOS POR SERVIDORES PÚBLICOS'
 'FISCALÍA DE INVESTIGACIÓN DE 

### Nota sobre agencia asociadas a los valores NaN:

La agencia TUR-2 corresponde a la Fiscalía de Investigación Territorial en Venustiano Carranza.

La agencia E corresponde a la FDS-5, que abarca la investigación de delitos ocurridos en Iztapalapa

In [4]:
print(AlcaldiaHecho_NaN.query(' agencia == "E" or agencia ==  "FDS-5" ').shape)
print("-----------------------------------")

#TUR2
secuestros.loc[secuestros['agencia'] == 'TUR-2', 'alcaldia_hecho'] = 'VENUSTIANO CARRANZA'

#E y FDS-5
secuestros.loc[(secuestros['agencia'] == 'E') | (secuestros['agencia'] == 'FDS-5'), 'alcaldia_hecho'] = 'IZTAPALAPA'

print(f'Ahora el número de valores faltantes en la columna "alcaldia_hecho" es: {secuestros['alcaldia_hecho'].isna().sum()}')

(75, 12)
-----------------------------------
Ahora el número de valores faltantes en la columna "alcaldia_hecho" es: 33


In [5]:
AlcaldiaHecho_NaN2=secuestros[pd.isna(secuestros['alcaldia_hecho'])] 
print("-----------------------------------")
print(pd.unique(AlcaldiaHecho_NaN2['fiscalia']))
print("-----------------------------------")
print(pd.unique(AlcaldiaHecho_NaN2['agencia']))

-----------------------------------
['INVESTIGACIÓN DE LOS DELITOS COMETIDOS POR SERVIDORES PUBLICOS'
 'INVESTIGACIÓN PARA LA ATENCIÓN DEL DELITO DE SECUESTRO'
 'INVESTIGACIÓN PARA LA ATENCIÓN DE NIÑOS, NIÑAS Y ADOLESCENTES'
 'INVESTIGACIÓN EN AGENCIAS DE ATENCIÓN ESPECIALIZADAS'
 'JUZGADOS FAMILIARES' 'AGENCIA CENTRAL DE INVESTIGACIÓN'
 'INVESTIGACIÓN PARA LA ATENCIÓN DEL DELITO DE HOMICIDIO'
 'FISCALÍA DE INVESTIGACIÓN DEL DELITO DE SECUESTRO'
 'FISCALÍA DE INVESTIGACIÓN DE DELITOS COMETIDOS EN AGRAVIO DE NIÑAS, NIÑOS Y ADOLESCENTES'
 'FISCALÍA DE INVESTIGACIÓN Y PERSECUCIÓN DE LOS DELITOS EN MATERIA DE DESAPARICIÓN FORZADA DE PERSONAS Y LA DESAPARICIÓN COMETIDA POR PARTICULARES Y BÚSQUEDA DE PERSONAS DESAPARECIDAS'
 'FISCALÍA DE INVESTIGACIÓN ESTRATÉGICA DE ASUNTOS ESPECIALES'
 'FISCALÍA DE INVESTIGACIÓN DE DELITOS COMETIDOS EN AGRAVIO DE GRUPOS DE ATENCIÓN PRIORITARIA'
 'FISCALÍA PARA LA INVESTIGACIÓN DE LOS DELITOS COMETIDOS POR SERVIDORES PÚBLICOS']
------------------------------

### **2.3 Sobre los 33 valores NaN restantes en 'alcaldia_hecho':**

Las agencias y fiscalias restantes abarcan desde dos alcaldías hasta las diesciseis alcaldías, por lo que al no encontrar otro modo de imputar los datos faltantes, siendo que estos representan menos del 0.5% de la columna mencionada, se ha tomado la decisión de eliminarlos.

In [6]:
print(f'El número de valores NaN restante en "alcaldia_hecho" representa el {round((AlcaldiaHecho_NaN2.shape[0] / secuestros.shape[0])*100, 2)}% de los valores de esta columna. ')
secuestros=secuestros.dropna(subset=['alcaldia_hecho'])
print("-----------------------------------")
print(secuestros.isna().sum())
print("-----------------------------------")
print(secuestros.shape)# **2. Proceso de imputación de valores faltantes para la alcaldía del hecho:**

El número de valores NaN restante en "alcaldia_hecho" representa el 0.49% de los valores de esta columna. 
-----------------------------------
anio_inicio               0
mes_inicio                0
fecha_inicio              1
fecha_hecho               5
hora_hecho                4
delito                    0
categoria_delito          0
fiscalia                  1
agencia                   0
unidad_investigacion      2
colonia_catalogo        542
alcaldia_hecho            0
dtype: int64
-----------------------------------
(6670, 12)


# **3. Exploración de otra ambigüedad en el dataframe:**

En los valores únicos sobre la columna 'alcaldia_hecho' aparece un string que se refiere a alcaldía indeterminada. Vamos a explorar este valor

In [7]:
print(pd.unique(secuestros['alcaldia_hecho']))
print("-----------------------------------")
print(f'El número de datos indeterminados en la columna "alcaldia_hecho" es: {secuestros.query(' alcaldia_hecho == "CDMX (indeterminada)" ').shape[0]}')
print("-----------------------------------")


#Dataframe de exploración
AlcaldiaHecho_indeterminada=secuestros.query(' alcaldia_hecho == "CDMX (indeterminada)" ')
print(pd.unique(AlcaldiaHecho_indeterminada['fiscalia']))
print("-----------------------------------")
print(pd.unique(AlcaldiaHecho_indeterminada['agencia']))

['BENITO JUAREZ' 'CUAUHTEMOC' 'MIGUEL HIDALGO' 'VENUSTIANO CARRANZA'
 'COYOACAN' 'IZTACALCO' 'IZTAPALAPA' 'TLALPAN' 'GUSTAVO A. MADERO'
 'TLAHUAC' 'ALVARO OBREGON' 'AZCAPOTZALCO' 'CUAJIMALPA DE MORELOS'
 'LA MAGDALENA CONTRERAS' 'XOCHIMILCO' 'MILPA ALTA' 'CDMX (indeterminada)']
-----------------------------------
El número de datos indeterminados en la columna "alcaldia_hecho" es: 14
-----------------------------------
['FISCALÍA PARA LA INVESTIGACIÓN DE LOS DELITOS COMETIDOS POR SERVIDORES PÚBLICOS'
 'FISCALÍA DE INVESTIGACIÓN Y PERSECUCIÓN DE LOS DELITOS EN MATERIA DE DESAPARICIÓN FORZADA DE PERSONAS Y LA DESAPARICIÓN COMETIDA POR PARTICULARES Y BÚSQUEDA DE PERSONAS DESAPARECIDAS'
 'FISCALÍA DE INVESTIGACIÓN TERRITORIAL EN CUAUHTÉMOC'
 'FISCALÍA DE INVESTIGACIÓN ESTRATÉGICA DE ASUNTOS ESPECIALES'
 'FISCALÍA DE INVESTIGACIÓN TERRITORIAL EN TLÁHUAC'
 'FISCALÍA DE INVESTIGACIÓN DEL DELITO DE SECUESTRO'
 'FISCALÍA DE INVESTIGACIÓN TERRITORIAL EN MIGUEL HIDALGO'
 'FISCALÍA DE INVESTIGACIÓ

### **3.1 Imputación de valores marcados como indeterminados en la columna 'alcaldia_hecho' mediante las columnas 'fiscalia' y 'agencia':**

In [8]:
#Cuauhtémoc 1
secuestros.loc[(secuestros['fiscalia'].str.contains('CUAU', case=False, na=False)) & (secuestros['alcaldia_hecho'] == \
                                                                                      'CDMX (indeterminada)'), 'alcaldia_hecho'] = 'CUAUHTEMOC'

#Cuauhtémoc 2
secuestros.loc[(secuestros['agencia'] == 'UAT-CUH-8') & (secuestros['alcaldia_hecho']=='CDMX (indeterminada)') , 'alcaldia_hecho'] = 'CUAUHTEMOC'


#Miguel Hidalgo
secuestros.loc[(secuestros['fiscalia'].str.contains('MIGUEL', case=False, na=False)) & (secuestros['alcaldia_hecho'] == \
                                                                                      'CDMX (indeterminada)'), 'alcaldia_hecho'] = 'MIGUEL HIDALGO'

#Tlahuac 1
secuestros.loc[(secuestros['fiscalia'].str.contains('HUAC', case=False, na=False)) & (secuestros['alcaldia_hecho'] == \
                                                                                      'CDMX (indeterminada)'), 'alcaldia_hecho'] = 'TLAHUAC'

#Tlahuac 2
secuestros.loc[(secuestros['agencia'] == 'UAT-TLH-1') & (secuestros['alcaldia_hecho']=='CDMX (indeterminada)') , 'alcaldia_hecho'] = 'TLAHUAC'


#Tlalpan 1
secuestros.loc[(secuestros['fiscalia'].str.contains('TLALPAN', case=False, na=False)) & (secuestros['alcaldia_hecho'] == \
                                                                                      'CDMX (indeterminada)'), 'alcaldia_hecho'] = 'TLALPAN'

#Tlalpan 2
secuestros.loc[(secuestros['agencia'] == 'UAT-TLP-5') & (secuestros['alcaldia_hecho']=='CDMX (indeterminada)') , 'alcaldia_hecho'] = 'TLALPAN'


print(f'Ahora el número de datos indeterminados en la columna "alcaldia_hecho" es: {secuestros.query(' alcaldia_hecho == "CDMX (indeterminada)" ').shape[0]}')

Ahora el número de datos indeterminados en la columna "alcaldia_hecho" es: 10


### **3.2 Sobre los 10 valores indeterminados restantes en 'alcaldia_hecho':**

Igualmente que como se mencionó con los datos nulos, agencias y fiscalias restantes abarcan desde dos alcaldías hasta las diesciseis alcaldías, por lo que al no encontrar otro modo de imputar la alcaldía ante estos datos indeterminados, se ha tomado la decisión de eliminarlos.

En total, en este dataframe se eliminaron 43 datos de la columna 'alcaldia_hecho', que originalmente tenía 6703. Esto es, se eliminó menos del 0.65% de datos que no se lograron imputar para el análisis exploratorio.

In [9]:
print(f'El porcentaje total de datos eliminados de la columna "alcaldia_hecho", contando datos NaN e indeterminados, es: {round((43/6703)*100,2)}% ')
secuestros=secuestros.query(' alcaldia_hecho != "CDMX (indeterminada)" ')


El porcentaje total de datos eliminados de la columna "alcaldia_hecho", contando datos NaN e indeterminados, es: 0.64% 


# **4. Trabajando los datetime data:**

Será útil darle formato a los años, ya que tenemos dos años que se asocian a la denuncia, el año en que fue puesta la denuncia y el año en que ocurrió el hecho. 

### **4.1 Convirtiendo fechaas a formato datetime64:**

In [10]:
#Fecha inicio:
secuestros['fecha_inicio'] = pd.to_datetime(secuestros['fecha_inicio'], format='mixed', errors='coerce')

#Fecha hecho:
secuestros['fecha_hecho'] = pd.to_datetime(secuestros['fecha_hecho'], format='mixed', errors='coerce')
print(secuestros.info())
print("-----------------------------------")
print(secuestros.isna().sum())

<class 'pandas.core.frame.DataFrame'>
Index: 6660 entries, 0 to 6702
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   anio_inicio           6660 non-null   int64         
 1   mes_inicio            6660 non-null   object        
 2   fecha_inicio          6659 non-null   datetime64[ns]
 3   fecha_hecho           6655 non-null   datetime64[ns]
 4   hora_hecho            6656 non-null   object        
 5   delito                6660 non-null   object        
 6   categoria_delito      6660 non-null   object        
 7   fiscalia              6659 non-null   object        
 8   agencia               6660 non-null   object        
 9   unidad_investigacion  6658 non-null   object        
 10  colonia_catalogo      6128 non-null   object        
 11  alcaldia_hecho        6660 non-null   object        
dtypes: datetime64[ns](2), int64(1), object(9)
memory usage: 676.4+ KB
None
----------

### **4.2 Ordenando por fecha del hecho:**

Posiblemente sea útil ordenar por fecha del hecho, para ver posibles relaciones también con otros datasets.

In [11]:
secuestros_ordenado=secuestros.sort_values(by='fecha_hecho', ascending=True)
display(secuestros_ordenado.head(2))
print("-----------------------------------")
display(secuestros_ordenado.tail(5))

,anio_inicio,mes_inicio,fecha_inicio,fecha_hecho,hora_hecho,delito,categoria_delito,fiscalia,agencia,unidad_investigacion,colonia_catalogo,alcaldia_hecho
6553,2024,Agosto,2024-08-31,1975-01-30,06:00:00,PRIVACION DE LA LIBERTAD PERSONAL,DELITO DE BAJO IMPACTO,FISCALÍA DE INVESTIGACIÓN TERRITORIAL EN ALVAR...,UAT-AO-4,UI-1SD,San Angel,ALVARO OBREGON
5744,2023,Enero,2023-01-20,1995-10-01,16:30:00,PRIVACION DE LA LIBERTAD PERSONAL,DELITO DE BAJO IMPACTO,FISCALÍA DE INVESTIGACIÓN Y PERSECUCIÓN DE LOS...,B,2 SIN DETENIDO,Bosque De Chapultepec I Seccion,MIGUEL HIDALGO


-----------------------------------


,anio_inicio,mes_inicio,fecha_inicio,fecha_hecho,hora_hecho,delito,categoria_delito,fiscalia,agencia,unidad_investigacion,colonia_catalogo,alcaldia_hecho
989,2016,Marzo,2016-03-02 00:00:00,NaT,NaN,PLAGIO O SECUESTRO,SECUESTRO,INVESTIGACIÓN PARA LA ATENCIÓN DEL DELITO DE S...,FAS,UI-3CD,Lindavista Norte,GUSTAVO A. MADERO
1107,2016,Abril,2016-04-29 00:00:00,NaT,NaN,PRIVACION DE LA LIBERTAD PERSONAL,DELITO DE BAJO IMPACTO,INVESTIGACIÓN EN AGENCIAS DE ATENCIÓN ESPECIAL...,TUR-2,UI-2SD,NaN,VENUSTIANO CARRANZA
3490,2019,Septiembre,2019-09-03 00:00:00,NaT,NaN,PRIVACION DE LA LIBERTAD PERSONAL,DELITO DE BAJO IMPACTO,INVESTIGACIÓN PARA LA ATENCIÓN DEL DELITO DE S...,E,3 CON DETENIDO 3 C/D,Pueblo Santiago Ahuizotla,IZTAPALAPA
4484,2020,Diciembre,2020-12-11 00:00:00,NaT,NaN,PRIVACION DE LA LIBERTAD PERSONAL,DELITO DE BAJO IMPACTO,FISCALÍA DE INVESTIGACIÓN DEL DELITO DE SECUESTRO,C,Unidad 14,NaN,CUAUHTEMOC
6661,2024,Diciembre,2024-12-20 16:27:50,NaT,NaT:00,PRIVACION DE LA LIBERTAD PERSONAL,DELITO DE BAJO IMPACTO,FISCALÍA DE INVESTIGACIÓN DEL DELITO DE SECUESTRO,E,3 CON DETENIDO 3 C/D,NaN,IZTAPALAPA


### **4.3 Creando una columna para el año del hecho:**


In [12]:
secuestros_ordenado['anio_hecho']=secuestros_ordenado['fecha_hecho'].dt.year.astype('Int64')
display(secuestros_ordenado.head(2))
print("-----------------------------------")
display(secuestros_ordenado[secuestros_ordenado['anio_hecho'].isna()])

,anio_inicio,mes_inicio,fecha_inicio,fecha_hecho,hora_hecho,delito,categoria_delito,fiscalia,agencia,unidad_investigacion,colonia_catalogo,alcaldia_hecho,anio_hecho
6553,2024,Agosto,2024-08-31,1975-01-30,06:00:00,PRIVACION DE LA LIBERTAD PERSONAL,DELITO DE BAJO IMPACTO,FISCALÍA DE INVESTIGACIÓN TERRITORIAL EN ALVAR...,UAT-AO-4,UI-1SD,San Angel,ALVARO OBREGON,1975
5744,2023,Enero,2023-01-20,1995-10-01,16:30:00,PRIVACION DE LA LIBERTAD PERSONAL,DELITO DE BAJO IMPACTO,FISCALÍA DE INVESTIGACIÓN Y PERSECUCIÓN DE LOS...,B,2 SIN DETENIDO,Bosque De Chapultepec I Seccion,MIGUEL HIDALGO,1995


-----------------------------------


,anio_inicio,mes_inicio,fecha_inicio,fecha_hecho,hora_hecho,delito,categoria_delito,fiscalia,agencia,unidad_investigacion,colonia_catalogo,alcaldia_hecho,anio_hecho
989,2016,Marzo,2016-03-02 00:00:00,NaT,NaN,PLAGIO O SECUESTRO,SECUESTRO,INVESTIGACIÓN PARA LA ATENCIÓN DEL DELITO DE S...,FAS,UI-3CD,Lindavista Norte,GUSTAVO A. MADERO,<NA>
1107,2016,Abril,2016-04-29 00:00:00,NaT,NaN,PRIVACION DE LA LIBERTAD PERSONAL,DELITO DE BAJO IMPACTO,INVESTIGACIÓN EN AGENCIAS DE ATENCIÓN ESPECIAL...,TUR-2,UI-2SD,NaN,VENUSTIANO CARRANZA,<NA>
3490,2019,Septiembre,2019-09-03 00:00:00,NaT,NaN,PRIVACION DE LA LIBERTAD PERSONAL,DELITO DE BAJO IMPACTO,INVESTIGACIÓN PARA LA ATENCIÓN DEL DELITO DE S...,E,3 CON DETENIDO 3 C/D,Pueblo Santiago Ahuizotla,IZTAPALAPA,<NA>
4484,2020,Diciembre,2020-12-11 00:00:00,NaT,NaN,PRIVACION DE LA LIBERTAD PERSONAL,DELITO DE BAJO IMPACTO,FISCALÍA DE INVESTIGACIÓN DEL DELITO DE SECUESTRO,C,Unidad 14,NaN,CUAUHTEMOC,<NA>
6661,2024,Diciembre,2024-12-20 16:27:50,NaT,NaT:00,PRIVACION DE LA LIBERTAD PERSONAL,DELITO DE BAJO IMPACTO,FISCALÍA DE INVESTIGACIÓN DEL DELITO DE SECUESTRO,E,3 CON DETENIDO 3 C/D,NaN,IZTAPALAPA,<NA>


# **5. Dataframe final:**

In [13]:
secuestros_ordenado.to_csv('df2_FGJ_CarpetasLibertad.csv', index=False) 